<a href="https://colab.research.google.com/github/SsHhKim/Deep-Learning-with-Colab/blob/master/Cifar_10_Keras_full_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import Activation, BatchNormalization
from keras.regularizers import l2
import numpy as np

Using TensorFlow backend.


In [2]:
(X_train, y_train0), (X_test, y_test0) = cifar10.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) int64


In [3]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

print(X_train.shape, X_train.dtype)

(50000, 32, 32, 3) float32


In [4]:
Y_train = tf.keras.utils.to_categorical(y_train0, 10)
Y_test = tf.keras.utils.to_categorical(y_test0, 10)

In [5]:
np.random.seed(0)
weight_decay = 1e-4

In [6]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

In [7]:
from keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [8]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [9]:
#data augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [10]:
from keras.callbacks import LearningRateScheduler

In [11]:
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                              steps_per_epoch=X_train.shape[0] // 64, epochs=50, 
                              verbose=2, validation_data=(X_test, Y_test), 
                              callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/50
 - 50s - loss: 1.9392 - accuracy: 0.4160 - val_loss: 1.2833 - val_accuracy: 0.5852
Epoch 2/50
 - 42s - loss: 1.2885 - accuracy: 0.5811 - val_loss: 1.1638 - val_accuracy: 0.6364
Epoch 3/50
 - 43s - loss: 1.0924 - accuracy: 0.6474 - val_loss: 1.0692 - val_accuracy: 0.6827
Epoch 4/50
 - 42s - loss: 0.9774 - accuracy: 0.6860 - val_loss: 0.8062 - val_accuracy: 0.7505
Epoch 5/50
 - 42s - loss: 0.9111 - accuracy: 0.7106 - val_loss: 0.8887 - val_accuracy: 0.7372
Epoch 6/50
 - 42s - loss: 0.8587 - accuracy: 0.7311 - val_loss: 0.8489 - val_accuracy: 0.7452
Epoch 7/50
 - 42s - loss: 0.8185 - accuracy: 0.7451 - val_loss: 0.7913 - val_accuracy: 0.7664
Epoch 8/50
 - 42s - loss: 0.7817 - accuracy: 0.7587 - val_loss: 0.7782 - val_accuracy: 0.7783
Epoch 9/50
 - 42s - loss: 0.7538 - accuracy: 0.7695 - val_loss: 0.7191 - val_accuracy: 0.7905
Epoch 10/50
 - 42s - loss: 0.7301 - accuracy: 0.7759 - val_loss: 0.6718 - val_accuracy: 0.8032
Epoch 11/50
 - 42s - loss: 0.7146 - accuracy: 0.7851 - val_

In [12]:
scores = model.evaluate(X_test, Y_test, batch_size=128, verbose=2)
print('\nAccuracy: %.4f' % scores[1])


Accuracy: 0.8724
